# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [1]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
from collections import defaultdict

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [2]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.letter_set = sorted(set().union(*self.full_dictionary))
        
        self.current_dictionary = []
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        self.incorrect_guesses = []
        self.probabilities = [0] * len(self.letter_set)
        checked_dictionary = self.full_dictionary
        self.used=False
        self.weight=1.0
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        occur=set(word)
        self.incorrect_guesses_till_now=list(set(self.guessed_letters)-occur)
        
        # If the last matched letter was wrong we replace remake the 4,3,2,1 grams
        if len(self.guessed_letters) != 0 and self.guessed_letters[-1] in self.incorrect_guesses_till_now :
            checked_dictionary = [
                word for word in self.full_dictionary 
                if not any(letter in self.incorrect_guesses_till_now for letter in word)
            ]
            self._1_gram =   self.build_1_gram(checked_dictionary)
            self._2_gram =   self.build_2_gram(checked_dictionary)
            self._3_gram =   self.build_3_gram(checked_dictionary) 
            self._4_gram =   self.build_4_gram(checked_dictionary)
            self._5_gram =   self.build_5_gram(checked_dictionary)
            self._6_gram =   self.build_6_gram(checked_dictionary)
            self._7_gram =   self.build_7_gram(checked_dictionary)
            self._8_gram =   self.build_8_gram(checked_dictionary)
            self._9_gram =   self.build_9_gram(checked_dictionary)
           
        
        self.probability = [0] * len(self.letter_set)
        
        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions (similar to WILDCARD MATCHING)
        self.used=False
        clean_word = word[::2].replace("_",".")
        self.weight=1.0
        
        return self.find_9_gram_probability(clean_word)
        
    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
               
    def find_9_gram_probability(self, word):
        letter_count = [0] * len(self.letter_set)
        probs = [0] * len(self.letter_set)
        total_count = 0
        j = 0
        self.weight = max(0, self.weight - 0.1 if self.used else self.weight)

        for i in range(len(word) - 8):
            
            # case 1: ".XXXXXXXXX"
            if word[i] == '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] != '.' and word[i+6] != '.' and word[i+7] != '.' and word[i+8] != '.':
                known_1 = word[i+1]
                known_2 = word[i+2]
                known_3 = word[i+3]
                known_4 = word[i+4]
                known_5 = word[i+5]
                known_6 = word[i+6]
                known_7 = word[i+7]
                known_8 = word[i+8]
                
                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._9_gram[letter][known_1][known_2][known_3][known_4][known_5][known_6][known_7][known_8] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._9_gram[letter][known_1][known_2][known_3][known_4][known_5][known_6][known_7][known_8]
                        letter_count[j] += self._9_gram[letter][known_1][known_2][known_3][known_4][known_5][known_6][known_7][known_8]

            # case 2: "X.XXXXXXX"
            if word[i] != '.' and word[i+1] == '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] != '.' and word[i+6] != '.' and word[i+7] != '.' and word[i+8] != '.':
                known_1 = word[i]
                known_2 = word[i+2]
                known_3 = word[i+3]
                known_4 = word[i+4]
                known_5 = word[i+5]
                known_6 = word[i+6]
                known_7 = word[i+7]
                known_8 = word[i+8]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._9_gram[known_1][letter][known_2][known_3][known_4][known_5][known_6][known_7][known_8] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._9_gram[known_1][letter][known_2][known_3][known_4][known_5][known_6][known_7][known_8]
                        letter_count[j] += self._9_gram[known_1][letter][known_2][known_3][known_4][known_5][known_6][known_7][known_8]

            # case 3: "XX.XXXXXX"
            if word[i] != '.' and word[i+1] != '.' and word[i+2] == '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] != '.' and word[i+6] != '.' and word[i+7] != '.' and word[i+8] != '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+3]
                known_4 = word[i+4]
                known_5 = word[i+5]
                known_6 = word[i+6]
                known_7 = word[i+7]
                known_8 = word[i+8]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._9_gram[known_1][known_2][letter][known_3][known_4][known_5][known_6][known_7][known_8] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._9_gram[known_1][known_2][letter][known_3][known_4][known_5][known_6][known_7][known_8]
                        letter_count[j] += self._9_gram[known_1][known_2][letter][known_3][known_4][known_5][known_6][known_7][known_8]

            # case 4: "XXX.XXXXX"
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] == '.' and word[i+4] != '.' and word[i+5] != '.' and word[i+6] != '.' and word[i+7] != '.' and word[i+8] != '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+2]
                known_4 = word[i+4]
                known_5 = word[i+5]
                known_6 = word[i+6]
                known_7 = word[i+7]
                known_8 = word[i+8]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._9_gram[known_1][known_2][known_3][letter][known_4][known_5][known_6][known_7][known_8] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._9_gram[known_1][known_2][known_3][letter][known_4][known_5][known_6][known_7][known_8]
                        letter_count[j] += self._9_gram[known_1][known_2][known_3][letter][known_4][known_5][known_6][known_7][known_8]

            # case 5: "XXXX.XXXX"
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] == '.' and word[i+5] != '.' and word[i+6] != '.' and word[i+7] != '.' and word[i+8] != '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+2]
                known_4 = word[i+3]
                known_5 = word[i+5]
                known_6 = word[i+6]
                known_7 = word[i+7]
                known_8 = word[i+8]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._9_gram[known_1][known_2][known_3][known_4][letter][known_5][known_6][known_7][known_8] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._9_gram[known_1][known_2][known_3][known_4][letter][known_5][known_6][known_7][known_8]
                        letter_count[j] += self._9_gram[known_1][known_2][known_3][known_4][letter][known_5][known_6][known_7][known_8]

            # case 6: "XXXXX.XXX"
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] == '.' and word[i+6] != '.' and word[i+7] != '.' and word[i+8] != '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+2]
                known_4 = word[i+3]
                known_4 = word[i+3]
                known_5 = word[i+4]
                known_6 = word[i+5]
                known_7 = word[i+6]
                known_8 = word[i+7]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._9_gram[known_1][known_2][known_3][known_4][known_5][known_6][letter][known_7][known_8] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._9_gram[known_1][known_2][known_3][known_4][known_5][known_6][letter][known_7][known_8]
                        letter_count[j] += self._9_gram[known_1][known_2][known_3][known_4][known_5][known_6][letter][known_7][known_8]

            # case 7: "XXXXXX.XX"
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] != '.' and word[i+6] == '.' and word[i+7] != '.' and word[i+8] != '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+2]
                known_4 = word[i+3]
                known_5 = word[i+4]
                known_6 = word[i+5]
                known_7 = word[i+7]
                known_8 = word[i+8]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._9_gram[known_1][known_2][known_3][known_4][known_5][known_6][known_7][letter][known_8] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._9_gram[known_1][known_2][known_3][known_4][known_5][known_6][known_7][letter][known_8]
                        letter_count[j] += self._9_gram[known_1][known_2][known_3][known_4][known_5][known_6][known_7][letter][known_8]

            # case 8: "XXXXXXX.X"
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] != '.' and word[i+6] != '.' and word[i+7] == '.' and word[i+8] != '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+2]
                known_4 = word[i+3]
                known_5 = word[i+4]
                known_6 = word[i+5]
                known_7 = word[i+6]
                known_8 = word[i+8]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._9_gram[known_1][known_2][known_3][known_4][known_5][known_6][known_7][known_8][letter] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._9_gram[known_1][known_2][known_3][known_4][known_5][known_6][known_7][known_8][letter]
                        letter_count[j] += self._9_gram[known_1][known_2][known_3][known_4][known_5][known_6][known_7][known_8][letter]

            # case 9: "XXXXXXX."
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] != '.' and word[i+6] != '.' and word[i+7] != '.' and word[i+8] == '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+2]
                known_4 = word[i+3]
                known_5 = word[i+4]
                known_6 = word[i+5]
                known_7 = word[i+6]
                known_8 = word[i+7]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._9_gram[known_1][known_2][known_3][known_4][known_5][known_6][known_7][known_8]['.'] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._9_gram[known_1][known_2][known_3][known_4][known_5][known_6][known_7][known_8]['.']
                        letter_count[j] += self._9_gram[known_1][known_2][known_3][known_4][known_5][known_6][known_7][known_8]['.']

        # calculate the probabilities of each letter appearing
        if total_count != 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count

        
        i = 0
        while i < len(self.probability):
            self.probability[i] = self.probability[i] + probs[i] * (self.weight)
            i += 1

        # run the next level down
        return self.find_8_gram_probability(word)
        
    def find_8_gram_probability(self, word):
        letter_count = [0] * len(self.letter_set)
        probs = [0] * len(self.letter_set)
        total_count = 0
        j = 0
        self.weight = max(0, self.weight - 0.1 if self.used else self.weight)

        for i in range(len(word) - 7):
            
            # case 1: ".XXXXXXX"
            if word[i] == '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] != '.' and word[i+6] != '.' and word[i+7] != '.':
                known_1 = word[i+1]
                known_2 = word[i+2]
                known_3 = word[i+3]
                known_4 = word[i+4]
                known_5 = word[i+5]
                known_6 = word[i+6]
                known_7 = word[i+7]
                
                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._8_gram[letter][known_1][known_2][known_3][known_4][known_5][known_6][known_7] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._8_gram[letter][known_1][known_2][known_3][known_4][known_5][known_6][known_7]
                        letter_count[j] += self._8_gram[letter][known_1][known_2][known_3][known_4][known_5][known_6][known_7]

            # case 2: "X.XXXXXX"
            if word[i] != '.' and word[i+1] == '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] != '.' and word[i+6] != '.' and word[i+7] != '.':
                known_1 = word[i]
                known_2 = word[i+2]
                known_3 = word[i+3]
                known_4 = word[i+4]
                known_5 = word[i+5]
                known_6 = word[i+6]
                known_7 = word[i+7]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._8_gram[known_1][letter][known_2][known_3][known_4][known_5][known_6][known_7] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._8_gram[known_1][letter][known_2][known_3][known_4][known_5][known_6][known_7]
                        letter_count[j] += self._8_gram[known_1][letter][known_2][known_3][known_4][known_5][known_6][known_7]

            # case 3: "XX.XXXXX"
            if word[i] != '.' and word[i+1] != '.' and word[i+2] == '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] != '.' and word[i+6] != '.' and word[i+7] != '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+3]
                known_4 = word[i+4]
                known_5 = word[i+5]
                known_6 = word[i+6]
                known_7 = word[i+7]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._8_gram[known_1][known_2][letter][known_3][known_4][known_5][known_6][known_7] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._8_gram[known_1][known_2][letter][known_3][known_4][known_5][known_6][known_7]
                        letter_count[j] += self._8_gram[known_1][known_2][letter][known_3][known_4][known_5][known_6][known_7]

            # case 4: "XXX.XXXX"
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] == '.' and word[i+4] != '.' and word[i+5] != '.' and word[i+6] != '.' and word[i+7] != '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+2]
                known_4 = word[i+4]
                known_5 = word[i+5]
                known_6 = word[i+6]
                known_7 = word[i+7]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._8_gram[known_1][known_2][known_3][letter][known_4][known_5][known_6][known_7] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._8_gram[known_1][known_2][known_3][letter][known_4][known_5][known_6][known_7]
                        letter_count[j] += self._8_gram[known_1][known_2][known_3][letter][known_4][known_5][known_6][known_7]

            # case 5: "XXXX.XXX"
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] == '.' and word[i+5] != '.' and word[i+6] != '.' and word[i+7] != '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+2]
                known_4 = word[i+3]
                known_5 = word[i+5]
                known_6 = word[i+6]
                known_7 = word[i+7]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._8_gram[known_1][known_2][known_3][known_4][letter][known_5][known_6][known_7] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._8_gram[known_1][known_2][known_3][known_4][letter][known_5][known_6][known_7]
                        letter_count[j] += self._8_gram[known_1][known_2][known_3][known_4][letter][known_5][known_6][known_7]

            # case 6: "XXXXX.XX"
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] == '.' and word[i+6] != '.' and word[i+7] != '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+2]
                known_4 = word[i+3]
                known_5 = word[i+4]
                known_6 = word[i+6]
                known_7 = word[i+7]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._8_gram[known_1][known_2][known_3][known_4][known_5][letter][known_6][known_7] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._8_gram[known_1][known_2][known_3][known_4][known_5][letter][known_6][known_7]
                        letter_count[j] += self._8_gram[known_1][known_2][known_3][known_4][known_5][letter][known_6][known_7]

            # case 7: "XXXXXX.X"
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] != '.' and word[i+6] == '.' and word[i+7] != '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+2]
                known_4 = word[i+3]
                known_5 = word[i+3]
                known_6 = word[i+4]
                known_7 = word[i+5]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._8_gram[known_1][known_2][known_3][known_4][known_5][known_6][letter][known_7] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._8_gram[known_1][known_2][known_3][known_4][known_5][known_6][letter][known_7]
                        letter_count[j] += self._8_gram[known_1][known_2][known_3][known_4][known_5][known_6][letter][known_7]

        # case 8: "XXXXXXX."
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] != '.' and word[i+6] != '.' and word[i+7] == '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+2]
                known_4 = word[i+3]
                known_5 = word[i+4]
                known_6 = word[i+5]
                known_7 = word[i+6]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._8_gram[known_1][known_2][known_3][known_4][known_5][known_6][known_7][letter] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._8_gram[known_1][known_2][known_3][known_4][known_5][known_6][known_7][letter]
                        letter_count[j] += self._8_gram[known_1][known_2][known_3][known_4][known_5][known_6][known_7][letter]

        # calculate the probabilities of each letter appearing
        if total_count != 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count

        # interpolate probabilities between _8_gram and _7_gram (adjust based on your model)
        i = 0
        while i < len(self.probability):
            self.probability[i] = self.probability[i] + probs[i] * (self.weight)
            i += 1

        # run the next level down
        return self.find_7_gram_probability(word)

    def find_7_gram_probability(self, word):
        letter_count = [0] * len(self.letter_set)
        probs = [0] * len(self.letter_set)
        total_count = 0
        j = 0
        self.weight = max(0, self.weight - 0.1 if self.used else self.weight)

        for i in range(len(word) - 6):
        
        # case 1: ".XXXXXX"
            if word[i] == '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] != '.' and word[i+6] != '.':
                known_1 = word[i+1]
                known_2 = word[i+2]
                known_3 = word[i+3]
                known_4 = word[i+4]
                known_5 = word[i+5]
                known_6 = word[i+6]
            
                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._7_gram[letter][known_1][known_2][known_3][known_4][known_5][known_6] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._7_gram[letter][known_1][known_2][known_3][known_4][known_5][known_6]
                        letter_count[j] += self._7_gram[letter][known_1][known_2][known_3][known_4][known_5][known_6]

        # case 2: "X.XXXXX"
            if word[i] != '.' and word[i+1] == '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] != '.' and word[i+6] != '.':
                known_1 = word[i]
                known_2 = word[i+2]
                known_3 = word[i+3]
                known_4 = word[i+4]
                known_5 = word[i+5]
                known_6 = word[i+6]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._7_gram[known_1][letter][known_2][known_3][known_4][known_5][known_6] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._7_gram[known_1][letter][known_2][known_3][known_4][known_5][known_6]
                        letter_count[j] += self._7_gram[known_1][letter][known_2][known_3][known_4][known_5][known_6]

        # case 3: "XX.XXXX"
            if word[i] != '.' and word[i+1] != '.' and word[i+2] == '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] != '.' and word[i+6] != '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+3]
                known_4 = word[i+4]
                known_5 = word[i+5]
                known_6 = word[i+6]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._7_gram[known_1][known_2][letter][known_3][known_4][known_5][known_6] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._7_gram[known_1][known_2][letter][known_3][known_4][known_5][known_6]
                        letter_count[j] += self._7_gram[known_1][known_2][letter][known_3][known_4][known_5][known_6]

        # case 4: "XXX.XXX"
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] == '.' and word[i+4] != '.' and word[i+5] != '.' and word[i+6] != '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+2]
                known_4 = word[i+4]
                known_5 = word[i+5]
                known_6 = word[i+6]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._7_gram[known_1][known_2][known_3][letter][known_4][known_5][known_6] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._7_gram[known_1][known_2][known_3][letter][known_4][known_5][known_6]
                        letter_count[j] += self._7_gram[known_1][known_2][known_3][letter][known_4][known_5][known_6]

        # case 5: "XXXX.XX"
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] == '.' and word[i+5] != '.' and word[i+6] != '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+2]
                known_4 = word[i+3]
                known_5 = word[i+5]
                known_6 = word[i+6]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._7_gram[known_1][known_2][known_3][known_4][letter][known_5][known_6] != 0 and letter not in    self.guessed_letters:
                        self.used = True
                        total_count += self._7_gram[known_1][known_2][known_3][known_4][letter][known_5][known_6]
                        letter_count[j] += self._7_gram[known_1][known_2][known_3][known_4][letter][known_5][known_6]

        # case 6: "XXXXX.X"
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] == '.' and word[i+6] != '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+2]
                known_4 = word[i+3]
                known_5 = word[i+4]
                known_6 = word[i+6]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._7_gram[known_1][known_2][known_3][known_4][known_5][letter][known_6] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._7_gram[known_1][known_2][known_3][known_4][known_5][letter][known_6]
                        letter_count[j] += self._7_gram[known_1][known_2][known_3][known_4][known_5][letter][known_6]

        # case 7: "XXXXXX."
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] != '.' and word[i+6] == '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+2]
                known_4 = word[i+3]
                known_5 = word[i+4]
                known_6 = word[i+5]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._7_gram[known_1][known_2][known_3][known_4][known_5][known_6][letter] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._7_gram[known_1][known_2][known_3][known_4][known_5][known_6][letter]
                        letter_count[j] += self._7_gram[known_1][known_2][known_3][known_4][known_5][known_6][letter]
        # calculate the probabilities of each letter appearing
        if total_count != 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count

        # interpolate probabilities between _6_gram and _5_gram (adjust based on your model)
        i = 0
        while i < len(self.probability):
            self.probability[i] = self.probability[i] + probs[i] * (self.weight)
            i += 1

        # run the next level down
        return self.find_6_gram_probability(word)

    def find_6_gram_probability(self, word):
        letter_count = [0] * len(self.letter_set)
        probs = [0] * len(self.letter_set)
        total_count = 0
        j = 0
        self.weight = max(0, self.weight - 0.1 if self.used else self.weight)

        for i in range(len(word) - 5):
            
            # case 1: ".XXXXX"
            if word[i] == '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] != '.':
                known_1 = word[i+1]
                known_2 = word[i+2]
                known_3 = word[i+3]
                known_4 = word[i+4]
                known_5 = word[i+5]
                
                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._6_gram[letter][known_1][known_2][known_3][known_4][known_5] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._6_gram[letter][known_1][known_2][known_3][known_4][known_5]
                        letter_count[j] += self._6_gram[letter][known_1][known_2][known_3][known_4][known_5]

            # case 2: "X.XXXX"
            if word[i] != '.' and word[i+1] == '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] != '.':
                known_1 = word[i]
                known_2 = word[i+2]
                known_3 = word[i+3]
                known_4 = word[i+4]
                known_5 = word[i+5]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._6_gram[known_1][letter][known_2][known_3][known_4][known_5] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._6_gram[known_1][letter][known_2][known_3][known_4][known_5]
                        letter_count[j] += self._6_gram[known_1][letter][known_2][known_3][known_4][known_5]

            # case 3: "XX.XXX"
            if word[i] != '.' and word[i+1] != '.' and word[i+2] == '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] != '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+3]
                known_4 = word[i+4]
                known_5 = word[i+5]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._6_gram[known_1][known_2][letter][known_3][known_4][known_5] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._6_gram[known_1][known_2][letter][known_3][known_4][known_5]
                        letter_count[j] += self._6_gram[known_1][known_2][letter][known_3][known_4][known_5]

            # case 4: "XXX.XX"
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] == '.' and word[i+4] != '.' and word[i+5] != '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+2]
                known_4 = word[i+4]
                known_5 = word[i+5]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._6_gram[known_1][known_2][known_3][letter][known_4][known_5] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._6_gram[known_1][known_2][known_3][letter][known_4][known_5]
                        letter_count[j] += self._6_gram[known_1][known_2][known_3][letter][known_4][known_5]

            # case 5: "XXXX.X"
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] == '.' and word[i+5] != '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+2]
                known_4 = word[i+3]
                known_5 = word[i+5]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._6_gram[known_1][known_2][known_3][known_4][letter][known_5] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._6_gram[known_1][known_2][known_3][known_4][letter][known_5]
                        letter_count[j] += self._6_gram[known_1][known_2][known_3][known_4][letter][known_5]

            # case 6: "XXXXX."
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.' and word[i+5] == '.':
                known_1 = word[i]
                known_2 = word[i+1]
                known_3 = word[i+2]
                known_4 = word[i+3]
                known_5 = word[i+4]

                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._6_gram[known_1][known_2][known_3][known_4][known_5][letter] != 0 and letter not in self.guessed_letters:
                        self.used = True
                        total_count += self._6_gram[known_1][known_2][known_3][known_4][known_5][letter]
                        letter_count[j] += self._6_gram[known_1][known_2][known_3][known_4][known_5][letter]

        # calculate the probabilities of each letter appearing
        if total_count != 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count

        # interpolate probabilities between _6_gram and _5_gram (adjust based on your model)
        i = 0
        while i < len(self.probability):
            self.probability[i] = self.probability[i] + probs[i] * (self.weight)
            i += 1

        # run the next level down
        return self.find_5_gram_probability(word)

    def find_5_gram_probability(self, word):
        
        letter_count = [0] * len(self.letter_set)
        probs = [0] * len(self.letter_set)
        total_count = 0
        j=0
        self.weight = max(0, self.weight - 0.1 if self.used else self.weight)

        # print("Running_5_gram")
        for i in range(len(word) - 4):
                        
            # case 3: ".XXXX "
            if word[i] == '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.':
                known_1= word[i+1]
                known_2= word[i+2]
                known_3= word[i+3]
                known_4= word[i+4]
                
                for j in range(len(self.letter_set)):
                    letter=self.letter_set[j]
                    try:
                        if self._4_gram[letter][known_1][known_2][known_3][known_4] != 0 and letter not in self.guessed_letters:
                            # print("Running_5.1_gram")
                            self.used=True
                            total_count += self._4_gram[letter][known_1][known_2][known_3][known_4]
                            letter_count[j] += self._4_gram[letter][known_1][known_2][known_3][known_4]
                    except:
                        continue
           # case 3: "X.XXX "
            if word[i] != '.' and word[i+1] == '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] != '.':
                known_1= word[i]
                known_2= word[i+2]
                known_3= word[i+3]
                known_4= word[i+4]
                
                for j in range(len(self.letter_set)):
                    letter=self.letter_set[j]
                    try:
                        if self._4_gram[known_1][letter][known_2][known_3][known_4] != 0 and letter not in self.guessed_letters:
                            # print("Running_5.2_gram")
                            self.used=True
                            total_count += self._4_gram[known_1][letter][known_2][known_3][known_4]
                            letter_count[j] += self._4_gram[known_1][letter][known_2][known_3][known_4]
                    except:
                        continue    
            # case 3: "XX.XX "
            if word[i] != '.' and word[i+1] != '.' and word[i+2] == '.' and word[i+3] != '.' and word[i+4] != '.':
                known_1= word[i]
                known_2= word[i+1]
                known_3= word[i+3]
                known_4= word[i+4]
                
                for j in range(len(self.letter_set)):
                    letter=self.letter_set[j]
                    try:

                        if self._4_gram[known_1][known_2][letter][known_3][known_4] != 0 and letter not in self.guessed_letters:
                            # print("Running_5.3_gram")
                            self.used=True
                            total_count += self._4_gram[known_1][known_2][letter][known_3][known_4]
                            letter_count[j] += self._4_gram[known_1][known_2][letter][known_3][known_4]
                    except:
                        continue    
            # case 3: "XXX.X "
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] == '.' and word[i+4] != '.':
                known_1= word[i]
                known_2= word[i+1]
                known_3= word[i+2]
                known_4= word[i+4]
                
                for j in range(len(self.letter_set)):
                    letter=self.letter_set[j]
                    try:
                        if self._4_gram[known_1][known_2][known_3][letter][known_4] != 0 and letter not in self.guessed_letters:
                            # print("Running_5.4_gram")
                            self.used=True
                            total_count += self._4_gram[known_1][known_2][known_3][letter][known_4]
                            letter_count[j] += self._4_gram[known_1][known_2][known_3][letter][known_4]
                    except:
                        continue    
            # case 5: "XXXX."
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.' and word[i+4] == '.':
                known_1= word[i]
                known_2= word[i+1]
                known_3= word[i+2]
                known_4= word[i+3]
                
                for j in range(len(self.letter_set)):
                    letter=self.letter_set[j]
                    try:
                        if self._4_gram[known_1][known_2][known_3][known_4][letter] != 0 and letter not in self.guessed_letters:
                            # print("Running_5.5_gram")
                            self.used=True
                            total_count += self._4_gram[known_1][known_2][known_3][known_4][letter]
                            letter_count[j] += self._4_gram[known_1][known_2][known_3][known_4][letter]
                    except:
                        continue          
        
        # calculate the probabilities of each letter appearing
        if total_count != 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count

        i=0
        while i < len(self.probability):
            self.probability[i] = self.probability[i] + probs[i] * (self.weight)
            i+=1
        
        # run the next level down
        return self.find_4_gram_probability(word)
    
    def find_4_gram_probability(self, word):
        
        letter_count = [0] * len(self.letter_set)
        probs = [0] * len(self.letter_set)
        total_count = 0
        j=0
        self.weight = max(0, self.weight - 0.1 if self.used else self.weight)

        for i in range(len(word) - 3):
                        
            # case 3: ".XXX "
            if word[i] == '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] != '.':
                known_1= word[i+1]
                known_2= word[i+2]
                known_3= word[i+3]
                
                for j in range(len(self.letter_set)):
                    letter=self.letter_set[j]
                    try:
                        if self._4_gram[letter][known_1][known_2][known_3] != 0 and letter not in self.guessed_letters:
                            # print("Running_4.1_gram")
                            self.used=True
                            total_count += self._4_gram[letter][known_1][known_2][known_3]
                            letter_count[j] += self._4_gram[letter][known_1][known_2][known_3]
                    except:
                        continue           
           # case 3: "X.XX "
            if word[i] != '.' and word[i+1] == '.' and word[i+2] != '.' and word[i+3] !='.':
                known_1= word[i]
                known_2= word[i+2]
                known_3= word[i+3]
                
                for j in range(len(self.letter_set)):
                    letter=self.letter_set[j]
                    try:
                        if self._4_gram[known_1][letter][known_2][known_3] != 0 and letter not in self.guessed_letters:
                            # print("Running_4.2_gram")
                            self.used=True
                            total_count += self._4_gram[known_1][letter][known_2][known_3]
                            letter_count[j] += self._4_gram[known_1][letter][known_2][known_3]
                    except:
                        continue    
            # case 3: "XX.X "
            if word[i] != '.' and word[i+1] != '.' and word[i+2] == '.' and word[i+3] !='.':
                known_1= word[i]
                known_2= word[i+1]
                known_3= word[i+3]
                
                for j in range(len(self.letter_set)):
                    letter=self.letter_set[j]
                    try:
                        if self._4_gram[known_1][known_2][letter][known_3] != 0 and letter not in self.guessed_letters:
                            # print("Running_4.3_gram")
                            self.used=True
                            total_count += self._4_gram[known_1][known_2][letter][known_3]
                            letter_count[j] += self._4_gram[known_1][known_2][letter][known_3]
                    except:
                        continue

            # case 3: "XXX. "
            if word[i] != '.' and word[i+1] != '.' and word[i+2] != '.' and word[i+3] =='.':
                known_1= word[i]
                known_2= word[i+1]
                known_3= word[i+2]
                
                for j in range(len(self.letter_set)):
                    letter=self.letter_set[j]
                    try:
                        if self._4_gram[known_1][known_2][known_3][letter] != 0 and letter not in self.guessed_letters:
                            # print("Running_4.3_gram")
                            self.used=True
                            total_count += self._4_gram[known_1][known_2][known_3][letter]
                            letter_count[j] += self._4_gram[known_1][known_2][known_3][letter]
                    except:
                        continue
        
        # calculate the probabilities of each letter appearing
        if total_count != 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count
        i=0
        while i < len(self.probability):
            self.probability[i] = self.probability[i] + probs[i] * (self.weight)
            i+=1
        
        # run the next level down
        return self.find_3_gram_probability(word)
    
    def find_3_gram_probability(self, word):
        
        letter_count = [0] * len(self.letter_set)
        probs = [0] * len(self.letter_set)
        total_count = 0
        j=0
        self.weight = max(0, self.weight - 0.1 if self.used else self.weight)

        for i in range(len(word) - 2):
                        
            # case 3: ".XX"
            if word[i] == '.' and word[i+1] != '.' and word[i+2] != '.':
                known_1 = word[i+1]
                known_2= word[i+2]
                
                for j in range(len(self.letter_set)):
                    letter=self.letter_set[j]
                    if self._3_gram[letter][known_1][known_2] != 0 and letter not in self.guessed_letters:
                        # print("Running_3.1_gram")
                        self.used=True
                        total_count += self._3_gram[letter][known_1][known_2]
                        letter_count[j] += self._3_gram[letter][known_1][known_2]
            
            # case 2: "X.X"
            if word[i] != '.' and word[i+1] == '.' and word[i+2] != '.':
                known_1 = word[i]
                known_2 = word[i+2]
                
                for j in range(len(self.letter_set)):
                    letter=self.letter_set[j]
                    if self._3_gram[known_1][letter][known_2] != 0 and letter not in self.guessed_letters:
                        # print("Running_3.2_gram")
                        self.used=True
                        total_count += self._3_gram[known_1][letter][known_2]
                        letter_count[j] += self._3_gram[known_1][letter][known_2]
                        
            # case 3: "XX."
            if word[i] != '.' and word[i+1] != '.' and word[i+2] == '.':
                known_1 = word[i]
                known_2 = word[i+1]
                
                for j in range(len(self.letter_set)):
                    letter=self.letter_set[j]
                    if self._3_gram[known_1][known_2][letter] != 0 and letter not in self.guessed_letters:
                        # print("Running_3.3_gram")
                        self.used=True
                        total_count += self._3_gram[known_1][known_2][letter]
                        letter_count[j] += self._3_gram[known_1][known_2][letter]
           
        
        # calculate the probabilities of each letter appearing
        if total_count != 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count
        
        # interpolate probabilities between _3_gram and _2_gram
        i=0
        while i < len(self.probability):
            self.probability[i] = self.probability[i] + probs[i] * (self.weight)
            i+=1
        
        # run the next level down
        return self.find_2_gram_probability(word)
    
    def find_2_gram_probability(self, word):
        
        letter_count = [0] * len(self.letter_set)
        probs = [0] * len(self.letter_set)
        total_count = 0
        j=0
        self.weight = max(0, self.weight - 0.1 if self.used else self.weight)


        for i in range(len(word) - 1):
            # case 1: ".X"
            if word[i] == '.' and word[i+1]!= '.':
                known = word[i+1]
                
                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._2_gram[len(word)][letter][known] != 0 and letter not in self.guessed_letters:
                        # print("Running_2.1_gram")
                        self.used=True
                        total_count += self._2_gram[len(word)][letter][known]
                        letter_count[j] += self._2_gram[len(word)][letter][known]
                        
            # case 2: "X."
            if word[i] != '.' and word[i+1] == '.':
                known = word[i]
                
                for j in range(len(self.letter_set)):
                    letter = self.letter_set[j]
                    if self._2_gram[len(word)][known][letter] != 0 and letter not in self.guessed_letters:
                        # print("Running_2.2_gram")
                        self.used=True
                        total_count += self._2_gram[len(word)][known][letter]
                        letter_count[j] += self._2_gram[len(word)][known][letter]
                  
                                                                    
        if total_count != 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count

        i = 0
        while i < len(self.probability):
            self.probability[i] = self.probability[i] + probs[i] * self.weight
            i += 1

        return self.find_1_gram_probability(word)
     
    def find_1_gram_probability(self, word):
    
        letter_count = [0] * len(self.letter_set)
        probs = [0] * len(self.letter_set)
        
        total_count = 0
        j=0
        self.weight = max(0, self.weight - 0.1 if self.used else self.weight)


        for i in range(len(word)):
            # case 1: ". only"
            if word[i] == '.':
                                
                while j < len(self.letter_set):
                    letter = self.letter_set[j]
                    if self._1_gram[len(word)][letter] != 0 and letter not in self.guessed_letters:
                        total_count += self._1_gram[len(word)][letter]
                        letter_count[j] += self._1_gram[len(word)][letter]
                    j += 1
                       
        if total_count != 0:
            for i in range(len(self.letter_set)):
                probs[i] = letter_count[i] / total_count
                
        i = 0
        while i < len(self.probability):
            self.probability[i] = self.probability[i] + probs[i] * self.weight
            i += 1
        
        max_prob = 0
        guess_letter = ''
        i = 0
        while i < len(self.letter_set):
            if self.probability[i] > max_prob:
                max_prob = self.probability[i]
                guess_letter = self.letter_set[i]
            i += 1
        
        # Put special focus on letters which have higher frequency in English Language Words (TOP 10) (E is highest)
        if guess_letter == '':
            letters = self.letter_set.copy()
            random.shuffle(letters)
            letters_random = letters + ['e','e','e','e','a','a','a','r','r','i','o','t','n','s','l','c'] 
            for letter in letters_random:
                if letter not in self.guessed_letters:
                    return letter
            
        return guess_letter
    
    
    def build_9_gram(self, dictionary):
    # We create a map-like structure for 9-grams having 10 entries for keys
        _9_gram = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(int)))))))))
    
        for word in dictionary:
            for i in range(len(word) - 8):
                _9_gram[word[i]][word[i+1]][word[i+2]][word[i+3]][word[i+4]][word[i+5]][word[i+6]][word[i+7]][word[i+8]] += 1

        return _9_gram

    def build_8_gram(self, dictionary):
    # We create a map-like structure for 8-grams having 9 entries for keys
        _8_gram = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(int))))))))
    
        for word in dictionary:
            for i in range(len(word) - 7):
                _8_gram[word[i]][word[i+1]][word[i+2]][word[i+3]][word[i+4]][word[i+5]][word[i+6]][word[i+7]] += 1

        return _8_gram

    def build_7_gram(self, dictionary):
    # We create a map-like structure for 7-grams having 8 entries for keys
        _7_gram = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(int)))))))
    
        for word in dictionary:
            for i in range(len(word) - 6):
                _7_gram[word[i]][word[i+1]][word[i+2]][word[i+3]][word[i+4]][word[i+5]][word[i+6]] += 1

        return _7_gram

    def build_6_gram(self, dictionary):
    # We create a map-like structure for 6-grams having 7 entries for keys
        _6_gram = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(int))))))
    
        for word in dictionary:
            for i in range(len(word) - 5):
                _6_gram[word[i]][word[i+1]][word[i+2]][word[i+3]][word[i+4]][word[i+5]] += 1

        return _6_gram

    def build_5_gram(self, dictionary):
        # We create  a map like structure for _5_grams having 6 entry for keys
        _5_gram = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(int)))))
        
        for word in dictionary:
            
            for i in range(len(word) - 4):
                _5_gram[word[i]][word[i+1]][word[i+2]][word[i+3]][word[i+4]] += 1

        return _5_gram
    
    def build_4_gram(self, dictionary):
        # We create  a map like structure for _4_grams having 5 entry for keys
        _4_gram = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(int))))
        
        for word in dictionary:
            
            for i in range(len(word) - 3):
                _4_gram[word[i]][word[i+1]][word[i+2]][word[i+3]] += 1

        return _4_gram
    
    def build_3_gram(self, dictionary):
        # We create  a map like structure for _3_grams having 4 entry for keys
        _3_gram = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
        
        for word in dictionary:
            
            for i in range(len(word) - 2):
                _3_gram[word[i]][word[i+1]][word[i+2]] += 1

        return _3_gram
    
    def build_2_gram(self, dictionary):
        # We create  a map like structure for _3_grams having 3 entry for keys
        _2_gram = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
        
        for word in dictionary:
            
            for i in range(len(word) - 1):
                _2_gram[len(word)][word[i]][word[i+1]] += 1

        return _2_gram
        
    def build_1_gram(self, dictionary):
        # We create  a map like structure for _3_grams having 2 entry for keys
        _1_gram = defaultdict(lambda: defaultdict(int))
        
        for word in dictionary:
            
            for i in (set(word)):
                _1_gram[len(word)][i] += 1

        return _1_gram

    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        self.incorrect_guesses = []
        self.probabilities = [0] * len(self.letter_set)
        checked_dictionary = self.full_dictionary
        self._1_gram =   self.build_1_gram(checked_dictionary)
        self._2_gram =   self.build_2_gram(checked_dictionary)
        self._3_gram =   self.build_3_gram(checked_dictionary)
        self._4_gram =   self.build_4_gram(checked_dictionary)
        self._5_gram =   self.build_5_gram(checked_dictionary)
        self._6_gram =   self.build_6_gram(checked_dictionary)
        self._7_gram =   self.build_7_gram(checked_dictionary)
        self._8_gram =   self.build_8_gram(checked_dictionary)
        self._9_gram =   self.build_9_gram(checked_dictionary)
       
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [3]:
api = HangmanAPI(access_token="e6faf727b786a23967aed6fe26199b", timeout=2000)


## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [22]:
api.start_game(practice=1,verbose=True)
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Successfully start a new game! Game ID: 0f1c765322e6. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': '0f1c765322e6', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ e _ _ _ _ _ _ _ e '}
Guessing letter: r
Sever response: {'game_id': '0f1c765322e6', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ e _ _ _ _ _ _ _ e '}
Guessing letter: n
Sever response: {'game_id': '0f1c765322e6', 'status': 'ongoing', 'tries_remains': 4, 'word': '_ e _ _ _ _ _ _ _ e '}
Guessing letter: l
Sever response: {'game_id': '0f1c765322e6', 'status': 'ongoing', 'tries_remains': 4, 'word': 'l e _ _ _ _ _ _ _ e '}
Guessing letter: s
Sever response: {'game_id': '0f1c765322e6', 'status': 'ongoing', 'tries_remains': 3, 'word': 'l e _ _ _ _ _ _ _ e '}
Guessing letter: d
Sever response: {'game_id': '0f1c765322e6', 'status': 'ongoing', 'tries_remains': 2, 'word': 'l e _ _ _ _ _ _ _ e '}
Guessing letter: t
Sever response: {'game_id': '0f1c765322e6', 'status': 'ong

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [5]:
# for i in range(1000):
    # print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    # api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    # time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [6]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.543
